In [1]:
import seaborn as sns
%matplotlib inline

sns.set_style('darkgrid')

from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

import numpy as np
import random
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import argparse
# Importing necssary modules
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot
from sklearn.metrics import classification_report, confusion_matrix
import math
import random
import warnings
from collections import Counter
# Custom script 
#%matplotlibe inline

KeyboardInterrupt: 

In [ ]:
# import urllib2
import os, sys
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn import feature_extraction
from sklearn import preprocessing
from random import seed, shuffle

# import utils as ut

SEED = 1234
seed(SEED)
np.random.seed(SEED)

def process_loaded_data(dataFrame):
    FEATURES_CLASSIFICATION = ["decile1b", "decile3", "lsat", "ugpa", "zfygpa","zgpa", "fulltime", "fam_inc", "sex", "race", "tier"]  # features to be used for classification
    CONT_VARIABLES = ["lsat", "ugpa", "zfygpa", "zgpa"]  # continuous features, will need to be handled separately from categorical features, categorical features will be encoded using one-hot
    CLASS_FEATURE = "y"  # the decision variable
    SENSITIVE_ATTRS = ["sex"]
    CAT_VARIABLES = ["decile1b", "decile3", "fulltime", "fam_inc", "sex", "race", "tier"]
    CAT_VARIABLES_INDICES = [1,2,7,8,9,10,11]
    # COMPAS_INPUT_FILE = "bank-full.csv"
    INPUT_FILE = "./datasets/law.csv"

    df = pd.read_csv(INPUT_FILE)
    
    # convert to np array
    data = df.to_dict('list')
    print(data.keys())
    for k in data.keys():
        data[k] = np.array(data[k])

    """ Feature normalization and one hot encoding """

    # convert class label 0 to -1
    y = data[CLASS_FEATURE]
    #print(np.unique(y))
    #y[y == 0] = 1
    #y[y==0] = -1
    #y[y == 1] = -1
    y = np.array([int(k) for k in y])

    X = np.array([]).reshape(len(y), 0)  # empty array with num rows same as num examples, will hstack the features to it
    
    x_control = defaultdict(list)
    i=0
    feature_names = []
    
    for attr in FEATURES_CLASSIFICATION:
        vals = data[attr]
        
        if attr in CONT_VARIABLES:
            vals = [float(v) for v in vals]
            vals = preprocessing.scale(vals)  # 0 mean and 1 variance
            vals = np.reshape(vals, (len(y), -1))  # convert from 1-d arr to a 2-d arr with one col
            

        else:  # for binary categorical variables, the label binarizer uses just one var instead of two
            lb = preprocessing.LabelEncoder()
            lb.fit(vals)
            vals = lb.transform(vals)
            vals = np.reshape(vals, (len(y), -1))
           
            #if attr == 'job':
            #    print(lb.classes_)
            #    print(lb.transform(lb.classes_))
            
           
            
            
        # add to sensitive features dict
        if attr in SENSITIVE_ATTRS:
            x_control[attr] = vals
            

        # add to learnable features
        X = np.hstack((X, vals))
        
        if attr in CONT_VARIABLES:  # continuous feature, just append the name
            feature_names.append(attr)
        else:  # categorical features
            if vals.shape[1] == 1:  # binary features that passed through lib binarizer
                feature_names.append(attr)
            else:
                for k in lb.classes_:  # non-binary categorical features, need to add the names for each cat
                    feature_names.append(attr + "_" + str(k))

    # convert the sensitive feature to 1-d array
    
    x_control = dict(x_control)
    
    for k in x_control.keys():
        assert (x_control[k].shape[1] == 1)  # make sure that the sensitive feature is binary after one hot encoding
        x_control[k] = np.array(x_control[k]).flatten()
    
    feature_names.append('target')
    
    return X, y, feature_names.index(SENSITIVE_ATTRS[0]), 1, x_control


def load_law():

    # COMPAS_INPUT_FILE = "bank-full.csv"
    COMPAS_INPUT_FILE = "./datasets/law.csv"

    df = pd.read_csv(COMPAS_INPUT_FILE)
    ranges = [0,30,40,50,100]
    mask = (df['fam_inc'] >=1) & (df['fam_inc'] <=2)
    df1 = df[mask]
    mask = (df['fam_inc'] ==3)
    df2 = df[mask]
    mask = (df['fam_inc'] >=4) & (df['fam_inc'] <=5)
    df3 = df[mask]
    df1 = df1.dropna()
    df2 = df2.dropna()
    df3 = df3.dropna()
    #df4['y'] = pd.Series(np.where(df4.y.values == 'yes', 1, 0),df4.index)
    #print(df4.groupby('y').count())
    X1,y1, sa_index, p_Group, x_control = process_loaded_data(df1)
    X2,y2, sa_index, p_Group, x_control = process_loaded_data(df2)
    X3,y3, sa_index, p_Group, x_control = process_loaded_data(df3)
    
    np_Group = 0
    return X1,X2,X3,y1,y2,y3, sa_index, p_Group, np_Group

X1,X2,X3,y1,y2,y3, sa_index, p_Group, np_Group = load_law()

# prepare train data and test data of each client
clients = {}
client_data_testx = []
client_data_testy = []
client_names = ['{}_{}'.format('client', i+1) for i in range(4)]

x_train, x_test, y_train, y_test = train_test_split(X1,y1,test_size=0.2)
Xtr1 = x_train
Xte1 = x_test
Ytr1 = y_train
Yte1 = y_test
Xtr = x_train
client_data_testx.append(Xte1)
client_data_testy.append(Yte1)
####
x_train, x_test, y_train, y_test = train_test_split(X2,y2,test_size=0.2)
Xtr2 = x_train
Xte2 = x_test
Ytr2 = y_train
Yte2 = y_test
client_data_testx.append(Xte2)
client_data_testy.append(Yte2)
####
x_train, x_test, y_train, y_test = train_test_split(X3,y3,test_size=0.2)
Xtr3 = x_train
Xte3 = x_test
Ytr3 = y_train
Yte3 = y_test
client_data_testx.append(Xte3)
client_data_testy.append(Yte3)
####


In [ ]:
#concatnate teset data
x_test_new = np.concatenate((client_data_testx[0], client_data_testx[1]), axis=0)
x_test_new = np.concatenate((x_test_new, client_data_testx[2]), axis=0)
y_test_new = np.concatenate((client_data_testy[0], client_data_testy[1]), axis=0)
y_test_new = np.concatenate((y_test_new, client_data_testy[2]), axis=0)
test_batched1 = tf.data.Dataset.from_tensor_slices((x_test_new, y_test_new)).batch(len(y_test_new))

In [ ]:
def create_clients(Xtr1, Ytr1,Xtr2,Ytr2,Xtr3,Ytr3,num_clients,initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''
    clients = {}
    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    data = list(zip(Xtr1, Ytr1))
    clients.update({client_names[0] :data})
    data = list(zip(Xtr2, Ytr2))
    clients.update({client_names[1] :data})
    data = list(zip(Xtr3, Ytr3))
    clients.update({client_names[2] :data})
    #data = list(zip(Xtr4, Ytr4))
    #clients.update({client_names[3] :data})

    return clients
clients = create_clients(Xtr1,Ytr1,Xtr2,Ytr2,Xtr3,Ytr3, num_clients=3, initial='client')

In [ ]:
#clients = create_clients(Xtr, Ytr, num_clients=10, initial='client')

In [ ]:
def batch_data(data_shard, bs=30):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [ ]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

#test_batched = tf.data.Dataset.from_tensor_slices((Xte, Yte)).batch(len(Yte))
#print('Number of client datasets: {l}'.format(l=len(test_batched)))

In [ ]:
print('Number of client datasets: {l}'.format(l=len(clients_batched)))
print('First dataset: {d}'.format(d=clients_batched['client_1']))

In [ ]:

client_names = list(clients_batched.keys())
bs = list(clients_batched[client_name])[0][0].shape[0] 
#first calculate the total training data points across clinets
global_count = sum([tf.data.experimental.cardinality(clients_batched[client_name]).numpy() for client_name in client_names])*bs
local_count = tf.data.experimental.cardinality(clients_batched['client_1']).numpy()*bs
print(global_count) 
print(local_count)

In [ ]:
print('Number of client datasets: {l}'.format(l=len(clients_batched)))
print('First dataset: {d}'.format(d=clients_batched['client_1']))
# check length of number of test datasets created for clients
#print('Number of client test datasets: {l}'.format(l=len(clients_test_data_batched)))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,InputLayer
from tensorflow.keras.callbacks import EarlyStopping
class SimpleMLP:
    @staticmethod
    def build(x_train,n):
        
        model = Sequential()
        model.add(InputLayer(input_shape=(x_train.shape[1],)))
        model.add(Dense(x_train.shape[1],activation='relu'))#,input_shape=(x_train.shape[1],)))
        #model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Dense(n*x_train.shape[1],activation='relu'))
        #model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(n*x_train.shape[1],activation='relu'))

        model.add(Dense(1,activation='sigmoid'))
        return model

In [ ]:
lr = 0.0001 
comms_round = 50
loss='binary_crossentropy'
metrics = ['accuracy']
optimizer = Adam(learning_rate=lr) 
'''
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )  
'''

In [ ]:
print(sa_index)
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad

def find_statistical_parity_score(data,labels,predictions,pp_Group=p_Group,npp_Group=np_Group):
    protected_pos = 0.
    protected_neg = 0.
    non_protected_pos = 0.
    non_protected_neg = 0.

    tp_protected = 0.
    tn_protected = 0.
    fp_protected = 0.
    fn_protected = 0.

    tp_non_protected = 0.
    tn_non_protected = 0.
    fp_non_protected = 0.
    fn_non_protected = 0.
    
    saIndex = sa_index
    saValue = pp_Group
    
    for idx, val in enumerate(data):
        # protected population
        if val[saIndex] == saValue:
            if predictions[idx] == 1:
                protected_pos += 1.
            else:
                protected_neg += 1.
            # correctly classified
            '''
            if labels[idx] == predictions[idx]:
                if labels[idx] == 1:
                    tp_protected += 1.
                else:
                    tn_protected += 1.
            # misclassified
            else:
                if labels[idx] == 1:
                    fn_protected += 1.
                else:
                    fp_protected += 1.
            '''
        else:
            if predictions[idx] == 1:
                non_protected_pos += 1.
            else:
                non_protected_neg += 1.
            '''
            # correctly classified
            if labels[idx] == predictions[idx]:
                if labels[idx] == 1:
                    tp_non_protected += 1.
                else:
                    tn_non_protected += 1.
            # misclassified
            else:
                if labels[idx] == 1:
                    fn_non_protected += 1.
                else:
                    fp_non_protected += 1.
            '''
    '''
    tpr_protected = tp_protected / (tp_protected + fn_protected)
    tnr_protected = tn_protected / (tn_protected + fp_protected)

    tpr_non_protected = tp_non_protected / (tp_non_protected + fn_non_protected)
    tnr_non_protected = tn_non_protected / (tn_non_protected + fp_non_protected)
    '''
    C_prot = (protected_pos) / (protected_pos + protected_neg)
    C_non_prot = (non_protected_pos) / (non_protected_pos + non_protected_neg)

    stat_par = C_non_prot - C_prot
    '''
    print("protected_pos: %s" %protected_pos)
    print("protected_neg: %s" %protected_neg)
    print("non-protected_pos: %s" %non_protected_pos)
    print("non-protected_neg: %s" %non_protected_neg)
    '''
    return stat_par
    
def find_eqop_score(data,labels,predictions, pp_Group=p_Group,npp_Group=np_Group):
    print("p_Group %s" % pp_Group)
    print("np_Group %s" % npp_Group)
    protected_pos = 0.
    protected_neg = 0.
    non_protected_pos = 0.
    non_protected_neg = 0.

    tp_protected = 0.
    tn_protected = 0.
    fp_protected = 0.
    fn_protected = 0.

    tp_non_protected = 0.
    tn_non_protected = 0.
    fp_non_protected = 0.
    fn_non_protected = 0.
    saIndex = sa_index
    saValue = pp_Group
    for idx, val in enumerate(data):
        # protrcted population
        if val[saIndex] == saValue:
            '''
            if predictions[idx] == 1:
                protected_pos += 1.
            else:
                protected_neg += 1.
            '''
            # correctly classified
            if labels[idx] == predictions[idx]:
                if labels[idx] == 1:
                    tp_protected += 1.
                #else:
                #    tn_protected += 1.
            # misclassified
            else:
                if labels[idx] == 1:
                    fn_protected += 1.
                #else:
                #    fp_protected += 1.

        else:
            '''
            if predictions[idx] == 1:
                non_protected_pos += 1.
            else:
                non_protected_neg += 1.
            '''
            # correctly classified
            if labels[idx] == predictions[idx]:
                if labels[idx] == 1:
                    tp_non_protected += 1.
                #else:
                #    tn_non_protected += 1.
            # misclassified
            else:
                if labels[idx] == 1:
                    fn_non_protected += 1.
                #else:
                #    fp_non_protected += 1.

    tpr_protected = tp_protected / (tp_protected + fn_protected)
    #tnr_protected = tn_protected / (tn_protected + fp_protected)

    tpr_non_protected = tp_non_protected / (tp_non_protected + fn_non_protected)
    #tnr_non_protected = tn_non_protected / (tn_non_protected + fp_non_protected)

    
    eqop = tpr_non_protected - tpr_protected
    return eqop

from sklearn.metrics import f1_score
def test_client_model(X_test, Y_test,  model,pp_Group=p_Group,npp_Group=np_Group):
    
    cce = tf.keras.losses.BinaryCrossentropy()
    logits = model.predict(X_test)
    preidect = np.around(logits)
    preidect = np.nan_to_num(preidect)
    Y_test = np.nan_to_num(Y_test)
    conf = (confusion_matrix(Y_test,preidect)) 
    TN = conf[0][0]
    FP = conf[0][1]
    FN = conf[1][0]
    TP = conf[1][1]
    sensitivity = TP/(TP+FN) 
    specificity = TN/(FP+TN)
        
    BalanceACC = (sensitivity+specificity)/2
    stat_parity = find_statistical_parity_score(X_test,Y_test,preidect, pp_Group, npp_Group)
    assigned_positive_labels = 0
    total_positive_labels = 0
    
    
    unique, counts = np.unique(preidect, return_counts=True)
    count_ap_dict = dict(zip(unique, counts))
    assigned_positive_labels = count_ap_dict.get(1,0)
    
    unique, counts = np.unique(Y_test, return_counts=True)
    count_tp_dict = dict(zip(unique, counts))
    total_positive_labels = count_tp_dict.get(1,0)
    
    '''
    for i in range(len(preidect)):
        if preidect[i]==1:
            assigned_positive_labels+=1
        if Y_test[i]==1:
            total_positive_labels+=1
    '''    
    #eqop = find_eqop_score(X_test,Y_test,preidect, pp_Group,npp_Group) 
    #print('eqop: {}'.format(eqop))
    #return eqop, assigned_positive_labels, total_positive_labels, BalanceACC
    print('stat_parity: {}'.format(stat_parity))
    
    return stat_parity, assigned_positive_labels, total_positive_labels, BalanceACC


def find_class_Weight(labels,majority_label,minority_label):
    unique, counts = np.unique(labels, return_counts=True)
    count_ap_dict = dict(zip(unique, counts))
    
    majority_class_weight = 1
    minority_class_weight = count_ap_dict.get(majority_label,0)/count_ap_dict.get(minority_label,1)
    class_weights={majority_label:1,minority_label:minority_class_weight}
    
    return class_weights

def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.BinaryCrossentropy()
    logits = model.predict(X_test)
    preidect = np.around(logits)
    preidect = np.nan_to_num(preidect)
    Y_test = np.nan_to_num(Y_test)
    stat_parity = find_statistical_parity_score(X_test,Y_test,preidect)
    eqop = find_eqop_score(X_test,Y_test,preidect)    
        
    conf = (confusion_matrix(Y_test,preidect))   
    loss = cce(Y_test, preidect)
    acc = accuracy_score(preidect,Y_test)
    print('comm_round: {} | global_acc: {} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss,conf,stat_parity, eqop

In [ ]:



# bulid K-SMOTE algorithem
# input (dmin, dmaj, K=number of data sample from dmin to upsampling, r=ratio of samples to reate in compare to dmaj)
# output(synthatic data for a client)
import random
from imblearn.over_sampling import SMOTENC

def k_nearest_neighbors(data, predict, k):
    #k=8
    #if len(data) >= k:
    #    warnings.warn('K is set to a value less than total voting groups!')

    distances = []
    count = 0
    for sample in data:
        euclidean_distance = np.linalg.norm(np.array(sample)-np.array(predict))
        distances.append([euclidean_distance,count])
        count+=1
    
    votes = [i[1] for i in sorted(distances)[:k]] ##votes is returning indexes of k random samples

    #vote_result = Counter(votes).most_common(9)[0][0]
    return votes


def fair_k_nearest_neighbors(data, predict, k):
    #k=8
    #if len(data) >= k:
    #    warnings.warn('K is set to a value less than total voting groups!')

    distances = []
    count = 0
    for sample in data:
        euclidean_distance = np.linalg.norm(np.array(sample)-np.array(predict))
        distances.append([euclidean_distance,count])
        count+=1
    
    votes = [i[1] for i in sorted(distances)[:k]] ##votes is returning indexes of k random samples

    #vote_result = Counter(votes).most_common(9)[0][0]
    return votes

##algo 2:

def fair_kSMOTE_algo_2(dmajor,dminor,k,r):
    S = []
    Ns =  int(r*(len(dmajor)))
    
    Nks = int(Ns / (k-1))
    difference = Ns-Nks*(k-1)
    
    rb = []
    #pick a random k sample from dmin and save them in rb
    dmin_rand = random.sample(dminor, k-1)   
    #for debugging
    sens_attr_vals = []
    i = 0
    
    #do algorithem (choose the nearest neighbor and linear interpolation)
    for xb in dmin_rand:
        N= k_nearest_neighbors(dminor,xb,k) #from minority-p
        
        #do linear interpolation
        Sxb = []
        
        if i==0:
            Nkss = Nks+difference
        else:
            Nkss = Nks
        
        i+=1
        for s in range(Nkss):
            
            
            j = 1  
            #randome k sample
            #j = random.randint(0, len(N))
            
            ##here nearst neghber insted of dminor
            #linear interpolation
            x_new = ((dminor[N[j]]-xb) * random.sample(range(0, 1), 1))
            j+=1
            
            while(j < len(N)):
                #here on random xb
                ind = N[j]
                x_new = x_new + ((dminor[ind]-xb) * random.sample(range(0, 1), 1))
                j += 1
                
            x_new = x_new / (len(N)-1) 
            Synthesized_instance = xb + x_new #
            
            
            #for algo 3 when finding nearest neighbors from min_np and assigning the 
            #'p' sensitive value to all synthesized instances
            Synthesized_instance[sa_index] = xb[sa_index] 
            Sxb.append(Synthesized_instance)
        
            
            
            
            
        S.append(Sxb)
    
    
    
    return S



###algo 3, algo 3 x_synthesized as a mixture of nearest neighbors from dmin-p and dmin-np

def fair_kSMOTE(dmajor,dminor_wg,dminor,k,r):
    S = []
    #Ns =  int(r*(len(dmajor) - len(dminor)))
    Ns =  int(r*(len(dmajor)))
    
    
    Nks = int(Ns / (k-1))
    difference = Ns-Nks*(k-1)
    #if r==-1:
    #    Nks = 1
    rb = []
    #pick a random k sample from dmin and save them in rb
    dmin_rand = random.sample(dminor, k-1)   
    #for debugging
    sens_attr_vals = []
    
    
    #do algorithem (choose the nearest neighbor and linear interpolation)
    i = 0
    
    for xb in dmin_rand:
        N= k_nearest_neighbors(dminor,xb,int(k/2)+1) #from minority-p
        N2= k_nearest_neighbors(dminor_wg,xb,int(k/2)) #from minority-np
    
        N3 = np.hstack((N, N2))
        if i==0:
            Nkss = Nks+difference
        else:
            Nkss = Nks
        
        i+=1
        #do linear interpolation
        Sxb = []
        
        for s in range(Nkss):
            
            j = 1  
            #randome k sample
            #j = random.randint(0, len(N))
            
            ##here nearst neghber insted of dminor
            #linear interpolation
            x_new = ((dminor[N[j]]-xb) * random.sample(range(0, 1), 1))
            j+=1
            
            while(j < len(N)):
                #here on random xb
                ind = N[j]
                
                x_new = x_new + ((dminor[ind]-xb) * random.sample(range(0, 1), 1))
                j += 1
            j = 0
            while(j < len(N2)):
                #here on random xb
                ind = N2[j]
                
                x_new = x_new + ((dminor_wg[ind]-xb) * random.sample(range(0, 1), 1))
                j += 1    
            x_new = x_new / (len(N3)-1) 
            Synthesized_instance = xb + x_new 
            
            
            #for algo 3 when finding nearest neighbors from min_np and assigning the 
            #'p' sensitive value to all synthesized instances
            Synthesized_instance[sa_index] = xb[sa_index] 
            Sxb.append(Synthesized_instance)
        
            
            
            
            
        S.append(Sxb)
    
    
   
    return S


def kSMOTE(dmajor,dminor,k,r):
    S = []
    Ns = int(r * (len(dmajor) - len(dminor)))
    Nks = int(Ns / k)
    rb = []
    #pick a random k sample from dmin and save them in rb
    dmin_rand = random.sample(dminor, k)   
    #for debugging
    sens_attr_vals = []
    
    
    #do algorithem (choose the nearest neighbor and linear interpolation)
    for xb in dmin_rand:
        N= k_nearest_neighbors(dminor,xb,k)
        
        
        #do linear interpolation
        Sxb = []
        for s in range(Nks):
            j = 1  
            #randome k sample
            #j = random.randint(0, len(N))
            
            ##here nearst neghber insted of dminor
            #linear interpolation
            x_new = ((dminor[N[j]]-xb) * random.sample(range(0, 1), 1))
            j+=1
            
            while(j < len(N)):
                #here on random xb
                ind = N[j]
                x_new = x_new + ((dminor[ind]-xb) * random.sample(range(0, 1), 1))
                j += 1
                
            x_new = x_new / len(N) 
            Synthesized_instance = xb + x_new 
            
            #for debugging
            sens_attr_vals.append(Synthesized_instance[10])
            
            #Synthesized_instance[sa_index] = xb[sa_index] ##Smote is synthesizing the values as if they are numbers but sensitive attribute 
                                                          ## is binary so we need to keep it binary
            Sxb.append(Synthesized_instance)
            
            
            
            
        S.append(Sxb)
    
    
    
    return S

def splitYtrain_smotenc(Xtr,Ytr,minority_lable):
  
    dmaj_x = []
    dmin_x = []
    label_maj_x = []
    label_min_x = []
    
    for i in range(len(Ytr)):
        if((Ytr[i]) == minority_lable):
            dmin_x.append(Xtr[i])
            label_min_x.append(Ytr[i])
        else:
            dmaj_x.append(Xtr[i])
            label_maj_x.append(Ytr[i])
    
    return dmaj_x,dmin_x, label_min_x, label_maj_x
def splitYtrain(Xtr,Ytr,minority_lable):
    #print(Ytr)
    dmaj_x = []
    dmin_x = []
    
    
    for i in range(len(Ytr)):
        if((Ytr[i]) == minority_lable):
            dmin_x.append(Xtr[i])
            
        else:
            dmaj_x.append(Xtr[i])
            
    
    return dmaj_x,dmin_x
def splitYtrain_sa_value(Xtr,Ytr,minority_lable,majority_label,pp_Group=p_Group,npp_Group=np_Group): #splite Ytrain based on sensitive attribute value
    #print(Ytr)
    dmaj_p_x = []
    dmaj_np_x = []
    dmin_p_x = []
    dmin_np_x = []
    
    
    for i in range(len(Ytr)):
        if((Ytr[i]) == minority_lable and (Xtr[i][sa_index])==pp_Group): #select minority instances with "protected" value 
            dmin_p_x.append(Xtr[i])
        elif((Ytr[i]) == minority_lable and (Xtr[i][sa_index])==npp_Group): #select minority instances with "protected" value 
            dmin_np_x.append(Xtr[i])
        elif ((Ytr[i]) == majority_label and (Xtr[i][sa_index])==pp_Group): #select minority(positive class) instances with "non-protected" value
            dmaj_p_x.append(Xtr[i])
        elif ((Ytr[i]) == majority_label and (Xtr[i][sa_index])==npp_Group): #select minority(positive class) instances with "non-protected" value
            dmaj_np_x.append(Xtr[i])
    
    return dmin_p_x, dmin_np_x, dmaj_p_x, dmaj_np_x
def get_statistics(Xtr,Ytr,minority_lable,majority_label): #splite Ytrain based on sensitive attribute value
    #print(Ytr)
    dmaj_p_x =0
    dmaj_np_x = 0
    dmin_p_x = 0
    dmin_np_x = 0
    
    
    for i in range(len(Ytr)):
        if((Ytr[i]) == minority_lable and (Xtr[i][sa_index])==p_Group): #select minority instances with "protected" value 
            dmin_p_x+=1
        elif((Ytr[i]) == minority_lable and (Xtr[i][sa_index])==np_Group): #select minority instances with "protected" value 
            dmin_np_x+=1
        elif ((Ytr[i]) == majority_label and (Xtr[i][sa_index])==p_Group): #select minority(positive class) instances with "non-protected" value
            dmaj_p_x+=1
        elif ((Ytr[i]) == majority_label and (Xtr[i][sa_index])==np_Group): #select minority(positive class) instances with "non-protected" value
            dmaj_np_x+=1
    
    return dmin_p_x, dmin_np_x, dmaj_p_x, dmaj_np_x
def create_synth_data(clinet_traning_x, clinet_traning_y, minority_lable,majority_label,k,r,group,pp_group=p_Group,npp_group=np_Group):
    
    
    
    #create two data set from traning data (one for maj (ex.0 class) and one for min(ex.1 class)) 
    #for simple federated learning
    dmaj_client,dmin_client = splitYtrain(clinet_traning_x,clinet_traning_y,minority_lable)
    
    #for fair federated learning
    
    
    dmin_p_x, dmin_np_x, dmaj_p_x, dmaj_np_x = splitYtrain_sa_value(clinet_traning_x,clinet_traning_y,minority_lable,majority_label, pp_group,npp_group)
    
    group_names = ['dmin_p_x', 'dmin_np_x', 'dmaj_p_x', 'dmaj_np_x']
    group_label_dict = {'dmin_p_x':minority_label, 'dmin_np_x': minority_label, 'dmaj_p_x':  majority_label, 'dmaj_np_x': majority_label}
    group_dict = {'dmin_p_x':dmin_p_x, 'dmin_np_x': dmin_np_x, 'dmaj_p_x':  dmaj_p_x, 'dmaj_np_x': dmaj_np_x}
    
    group_lengths = [len(dmin_p_x),len(dmin_np_x), len(dmaj_p_x), len(dmaj_np_x)] 
    
    #group with maximum length
    max_length_group = group_lengths.index(max(group_lengths))  
    
    #group name which has maximum length
    max_group_name = group_names[max_length_group]
    
    #find the insances in the group with maximum length and store them in dmaj_x
    for key, value in group_dict.items():
        if key== max_group_name:
            dmaj_x = value
            break
    Xtr_new = []
    Ytr_new = []  
    
    
    
    
    ##Algo 3:
    
     ##Algo 3:
    if group =='min_p':
        dmaj_x = dmaj_p_x
        dmin_x = dmin_p_x
        
        #dmin_np = dmin_np_x
        x_syn = fair_kSMOTE(dmaj_x,dmin_np_x,dmin_x,k,r)
        #x_syn = fair_kSMOTE_algo_2(dmaj_x,dmin_x,k,r)
        # add the created synthatic data to the traning data
        # here merrge old traning data with the new synthatic data
        new_label = minority_label
        for j in x_syn:
            for s in j:
                Xtr_new.append(s)
                Ytr_new.append(new_label)
        
    elif group =='maj_np':
        dmaj_x = dmin_np_x
        dmin_x = dmaj_np_x
        #dmin_np = dmin_np_x
        x_syn = fair_kSMOTE_algo_2(dmaj_x,dmin_x,k,r)
        
        
        # add the created synthatic data to the traning data
        # here merrge old traning data with the new synthatic data
        new_label = majority_label
        for j in x_syn:
            for s in j:
                Xtr_new.append(s)
                Ytr_new.append(new_label)
    
    
          
   
    
    return Xtr_new,Ytr_new
def k_nearest_neighbors_modified(data, predict, k,data_rand_index):
    distances = []
    count=0
    for sample in data:
        if count in data_rand_index:
            count+=1
        else:
            euclidean_distance = np.linalg.norm(np.array(sample)-np.array(predict))
            distances.append([euclidean_distance,count])
            count+=1
    votes = [i[1] for i in sorted(distances)[:k]] ##votes is returning indexes of k random samples
    #vote_result = Counter(votes).most_common(9)[0][0]
    return votes
def downsample_utility_function(clinet_traning_x, clinet_traning_y,data,data_index, k,r,label,reduction_amount):
    S = []
    #pick a random k sample from data which we want to downsample
    d_rand = []
    d_rand_index = []
    
    #reduction_amount = int(r*len(data))  
    #print("reduction amount: %s" % reduction_amount)
    
    for i in range(reduction_amount):
        index = random.randint(0, (len(data)-1))
        #random.sample(range(0,len(data)),1)
        
        d_rand.append(data[index])
        d_rand_index.append(index)
    data_rand_index = list(d_rand_index)
    Sxb = []   
    sens_attr_vals = []
    k = 3 ##number of nearest neighbours
    indices_neighbours_removed = list(d_rand_index)
    #print(len(d_rand_index))
    #do algorithm (choose the nearest neighbor and linear interpolation)
    
    Nks=1
    S=[]
    #do algorithem (choose the nearest neighbor and linear interpolation)
    for xb in d_rand:
        #N= k_nearest_neighbors(data,xb,k)
        N= k_nearest_neighbors_modified(data,xb,k,data_rand_index)
        
        #do linear interpolation
        Sxb = []
        for s in range(Nks):
            j = 0 #j = 1
            #linear interpolation
            x_new = ((data[N[j]]-xb) * random.sample(range(0, 1), 1))
            indices_neighbours_removed.append(N[1])
            j+=1
            
            while(j < len(N)-1):
                #here on random xb
                ind = N[j]
                x_new = x_new + ((data[ind]-xb) * random.sample(range(0, 1), 1))
                j += 1
                
            x_new = x_new / (len(N)-1) 
            Synthesized_instance = xb + x_new 
            Synthesized_instance[sa_index] = xb[sa_index] 
            
            Sxb.append(Synthesized_instance)
            
            
            
            
        S.append(Sxb)
    #print(indices_neighbours_removed)
    
    
    for i in range(len(indices_neighbours_removed)):
        indx = indices_neighbours_removed[i]
        indices_neighbours_removed[i] = data_index[indx]
    indices = np.unique(indices_neighbours_removed)
    indices = list(indices)
    difference = (reduction_amount*2)-len(indices)
    
        
    #for i in range(0,difference-1):
    #    indices.pop(0)
    #print("reduced amount: %s" % len(indices))
    
    indices.sort()
    
    Xtr = []
    Ytr = []
    for k in clinet_traning_x:
        Xtr.append(k)
        
    for k in clinet_traning_y:
        Ytr.append(k)
    
    for i in range(len(indices)):
        indx = indices[i]-i
        Xtr.pop(indx)
        Ytr.pop(indx)
    
    for j in S:
            for s in j:
                Xtr.append(s)
                Ytr.append(label)
    '''
    indices=[]
    for i in range(difference):
        index = random.randint(0, (len(Xtr)-1))
        indices.append(index)
    indices.sort()
    
    for i in range(len(indices)):
        indx = indices[i]-i
        Xtr.pop(indx)
        Ytr.pop(indx)
    '''    
    Xtr = np.array(Xtr)
    Ytr = np.array(Ytr)
    return Xtr, Ytr
def splitYtrain_sa_value_index(Xtr,Ytr,minority_lable,majority_label,pp_Group=p_Group,npp_Group=np_Group): #split data based on sensitive attribute value
    #print(Ytr)
    dmaj_p_x = []
    dmaj_p_index = [] ##index of maj_p instance in the main dataset
    
    dmaj_np_x = []
    dmaj_np_index = [] ##index of maj_np instance in the main dataset
    
    dmin_p_x = []
    dmin_p_index = [] ##index of min_p instance in the main dataset
    
    dmin_np_x = []
    dmin_np_index = [] ##index of min_np instance in the main dataset
    
    
    for i in range(len(Ytr)):
        if((Ytr[i]) == minority_lable and (Xtr[i][sa_index])==pp_Group): #select minority instances with "protected" value 
            dmin_p_x.append(Xtr[i])
            dmin_p_index.append(i)
        elif((Ytr[i]) == minority_lable and (Xtr[i][sa_index])==npp_Group): #select minority instances with "protected" value 
            dmin_np_x.append(Xtr[i])
            dmin_np_index.append(i)
        elif ((Ytr[i]) == majority_label and (Xtr[i][sa_index])==pp_Group): #select minority(positive class) instances with "non-protected" value
            dmaj_p_x.append(Xtr[i])
            dmaj_p_index.append(i)
        elif ((Ytr[i]) == majority_label and (Xtr[i][sa_index])==npp_Group): #select minority(positive class) instances with "non-protected" value
            dmaj_np_x.append(Xtr[i])
            dmaj_np_index.append(i)
    
    return dmin_p_x, dmin_np_x, dmaj_p_x, dmaj_np_x, dmin_p_index, dmin_np_index, dmaj_p_index, dmaj_np_index
def splitYtrain_sa_value_index(Xtr,Ytr,minority_lable,majority_label,pp_Group=p_Group,npp_Group=np_Group): #split data based on sensitive attribute value
    #print(Ytr)
    dmaj_p_x = []
    dmaj_p_index = [] ##index of maj_p instance in the main dataset
    
    dmaj_np_x = []
    dmaj_np_index = [] ##index of maj_np instance in the main dataset
    
    dmin_p_x = []
    dmin_p_index = [] ##index of min_p instance in the main dataset
    
    dmin_np_x = []
    dmin_np_index = [] ##index of min_np instance in the main dataset
    
    
    for i in range(len(Ytr)):
        if((Ytr[i]) == minority_lable and (Xtr[i][sa_index])==pp_Group): #select minority instances with "protected" value 
            dmin_p_x.append(Xtr[i])
            dmin_p_index.append(i)
        elif((Ytr[i]) == minority_lable and (Xtr[i][sa_index])==npp_Group): #select minority instances with "protected" value 
            dmin_np_x.append(Xtr[i])
            dmin_np_index.append(i)
        elif ((Ytr[i]) == majority_label and (Xtr[i][sa_index])==pp_Group): #select minority(positive class) instances with "non-protected" value
            dmaj_p_x.append(Xtr[i])
            dmaj_p_index.append(i)
        elif ((Ytr[i]) == majority_label and (Xtr[i][sa_index])==npp_Group): #select minority(positive class) instances with "non-protected" value
            dmaj_np_x.append(Xtr[i])
            dmaj_np_index.append(i)
    
    return dmin_p_x, dmin_np_x, dmaj_p_x, dmaj_np_x, dmin_p_index, dmin_np_index, dmaj_p_index, dmaj_np_index

def downsample(clinet_traning_x, clinet_traning_y, minority_lable,majority_label,k,r,group,pp_group=p_Group,npp_group=np_Group):
    
    #group: maj_p, maj_np, min_p, min_np
    
    dmaj_client,dmin_client = splitYtrain(clinet_traning_x,clinet_traning_y,minority_lable)
   
    #for fair federated learning
    
    dmin_p_x, dmin_np_x, dmaj_p_x, dmaj_np_x, dmin_p_index, dmin_np_index, dmaj_p_index, dmaj_np_index = splitYtrain_sa_value_index(clinet_traning_x,clinet_traning_y,minority_lable,majority_label,pp_group,npp_group)
    
    ##Algo 5:
    if group =='min_np':
        
        label = minority_label
        data = dmin_np_x
        reduction_amount = int(r*len(data))
        #print("reduction amount: %s" % reduction_amount)
        x_small,y_small = downsample_utility_function(clinet_traning_x, clinet_traning_y,data,dmin_np_index, k,r,label,reduction_amount)
        
        #difference = len(clinet_traning_x) - len(x_small)
        #if difference+2<reduction_amount:
        #    reduction_amount = reduction_amount-difference
        #    x_small,y_small = downsample_utility_function(x_small,y_small,data,dmin_np_index, k,r,label,reduction_amount)
        
        
        # add the created synthatic data to the traning data
        # here merrge old traning data with the new synthatic data
        #new_label = minority_label
        #for j in x_small:
        #    for s in j:
        #        Xtr_new.append(s)
        #        Ytr_new.append(new_label)
        
    elif group =='maj_p':
        label = majority_label
        data = dmaj_p_x
        reduction_amount = int(r*len(data))
        #print("reduction amount: %s" % reduction_amount)
        x_small,y_small = downsample_utility_function(clinet_traning_x, clinet_traning_y,data,dmaj_p_index, k,r,label,reduction_amount)
        # add the created synthatic data to the traning data
        # here merrge old traning data with the new synthatic data
        #difference = len(clinet_traning_x) - len(x_small)
        #if difference+2<reduction_amount:
        #    reduction_amount = reduction_amount-difference
        #    x_small,y_small = downsample_utility_function(x_small,y_small,data,dmin_np_index, k,r,label,reduction_amount)
    return x_small,y_small
def create_synth_data_from_smote_nc(clinet_traning_x, clinet_traning_y, minority_lable,majority_label,k,r):
    #create two data set from traning data (one for maj (ex.0 class) and one for min(ex.1 class)) 
    
    #for simple federated learning
    #dmaj_x,dmin_x = splitYtrain(clinet_traning_x,clinet_traning_y,minority_lable)
    
    #for fair federated learning
    CAT_VARIABLES_INDICES = [1,2,3,4,6,7,8,10,14,15]
    dmaj_x,dmin_x,label_min_x,label_maj_x = splitYtrain_smotenc(clinet_traning_x,clinet_traning_y,minority_lable)
    smote_nc = SMOTENC(categorical_features=CAT_VARIABLES_INDICES, sampling_strategy = r, random_state=0)
    X_resampled, y_resampled = smote_nc.fit_resample(clinet_traning_x, clinet_traning_y)
    
    return X_resampled,y_resampled

In [ ]:
smlp_global = SimpleMLP()
num_layers_mult=1
n=num_layers_mult
comms_round = 50
global_model = smlp_global.build(Xtr,n)
sensitivity_= []
specificity_= []
BalanceACC_= []
G_mean_= []
FP_rate_= []
FN_rate_= []
assigned_positives = 0
total_positives = 0
accuracy_= []
loss_= []
statistical_parity_ = []
eqop_ = []
disc_thresh = 0.005
lambda_initial = 0.005
disc_tolerance = 0.1
epsilon=1
pp_group = p_Group
npp_group=np_Group
#commence global training loop
import time

start = time.time()
end_time = []
e=EarlyStopping(patience=5,restore_best_weights=True)
from imblearn.over_sampling import SMOTE

for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    #random.shuffle(client_names)
    client_number=0
    #loop through each client and create new local model
    for client in client_names:   
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(Xtr, n)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #here we should get the traning data from clients and try to do ksmote and see changes
        count=0
        xxte = [] 
        yte = []
        
        
        for(X_test, Y_test) in clients_batched[client]:
            
            i = 0
            while (i <len(X_test)):       
                xxte.append(X_test[i].numpy())  
                yte.append(Y_test[i].numpy())
                i +=1
        
        ## separate particular clients test data from clients_test_data_batched
        #x_test_client = []
        #y_test_client = []
        #while(i < len(client_data_testx)):
        y_test_client = client_data_testy[client_number]
        x_test_client = client_data_testx[client_number]
        test_batched = tf.data.Dataset.from_tensor_slices((x_test_client, y_test_client)).batch(len(y_test_client))
        #print(x_test_client[0])
        client_number+=1    
        #here we change k and r to see how affect our result
        #for simple federated learning
        #Xtr_1_new,Ytr_1_new = create_synth_data(xxte,yte, 1,8,0.2)
        
        
        
        #for fair federated learning
        minority_label = 1
        majority_label = 0
        #lambda_score = 0.2 #0.1,0.15,0.2,0.25,0.3
        print("bismillah")
        print(client)
        
        ###
        prev_xtr,prev_ytr=[],[]
        prev_disc_score= []
        ###
        
        if comm_round == 0:
            Xtr_1_new=np.array(xxte)
            Ytr_1_new=np.array(yte)
            
            Xtr_1_new = np.array(Xtr_1_new)
            Ytr_1_new = np.array(Ytr_1_new)
            added_points = len(Xtr_1_new) - len(xxte)
            
            #new batch for new data
            #split data for validation set
            Xtr_1_new_split,x_val,Ytr_1_new_split,y_val=train_test_split(Xtr_1_new,Ytr_1_new,test_size=0.10,shuffle=True)
        
            data = list(zip(Xtr_1_new_split, Ytr_1_new_split))
            random.shuffle(data)
            btach_data = batch_data(data, bs=30)  
           
            class_weights=find_class_Weight(Ytr_1_new, majority_label,minority_label)
            local_model.fit(btach_data,validation_data=(x_val,y_val),validation_steps=1, callbacks=[e], epochs=3, verbose=0, 
                            class_weight=class_weights)
            ## find disc score for each client
            disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model)
            print("balanced_accuracy_client %s" % balanced_accuracy_client)
            if disc_score<0:
                pp_group = np_Group
                npp_group=p_Group
                disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model,pp_group,npp_group)
            else:
                pp_group = p_Group
                npp_group=np_Group
            
            
            trade_off = (1+epsilon**2)*((balanced_accuracy_client*(1-abs(disc_score)))/(epsilon*balanced_accuracy_client+(1-abs(disc_score))))
            lambda_score = lambda_initial*(1+(disc_score/disc_tolerance))
            
        
        if comm_round!=0:
            disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model,1,0)
            if disc_score<0:
                pp_group = np_Group
                npp_group=p_Group
                disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model,pp_group,npp_group)
            else:
                pp_group = p_Group
                npp_group=np_Group
            trade_off = (1+epsilon**2)*((balanced_accuracy_client*(1-abs(disc_score)))/(epsilon*balanced_accuracy_client+(1-abs(disc_score))))
        #Xtr_1_new, Ytr_1_new = xxte, yte 
        if disc_score > disc_thresh:
            
        
                prev_xtr,prev_ytr=[],[]
                prev_disc_score= []
                prev_trade_off=[]
                
                lambda_score = lambda_initial*(1+(disc_score/disc_tolerance))
                greater_disc_score = 0
                min_disc_score =disc_score
                max_trade_off = trade_off
                Xtr_1_new=np.array(xxte)
                Ytr_1_new=np.array(yte)
                prev_xtr,prev_ytr = Xtr_1_new, Ytr_1_new
                
               
                class_weights=find_class_Weight(Ytr_1_new, majority_label,minority_label)
                points = len(Xtr_1_new) - len(xxte)
                itr = 0
                
                while disc_score > disc_thresh:
                    
                    
                    itr+=1
                    closest_to_zero = min(disc_score, min_disc_score, key=abs)
                    if trade_off>max_trade_off:
                    #if closest_to_zero != min_disc_score:
                        min_disc_score = disc_score
                        max_trade_off = trade_off
                        prev_xtr,prev_ytr = Xtr_1_new, Ytr_1_new
                        points = len(Xtr_1_new) - len(xxte)
                        local_model.save_weights('./checkpoints/law2/income-eqop/my_checkpoint')
                    if assigned_positives<= total_positives:
                        
                        
                        #N(C+,S−) =N(C+,S−)+λ× N(C−,S−)
                        Xtr_min_p_new,Ytr_min_p_new = create_synth_data(xxte, yte, minority_label,majority_label,5,lambda_score,'min_p',pp_group,npp_group)
                    
                        #N(C−,S−) =N(C−,S−)−λ× N(C−,S−)
                        Xtr_maj_p_new,Ytr_maj_p_new = downsample(xxte, yte, minority_label,majority_label,5,lambda_score,'maj_p',pp_group,npp_group)    
                        
                        for k in Xtr_maj_p_new:
                            Xtr_min_p_new.append(k)
                        for k in Ytr_maj_p_new:
                            Ytr_min_p_new.append(k)
                        
                        Xtr_1_new = np.array(Xtr_min_p_new)
                        Ytr_1_new = np.array(Ytr_min_p_new)
                        
                        
                    else:
                        
                        #N(C−,S+) =N(C−,S+)+λ× N(C+,S+)
                        Xtr_maj_np_new,Ytr_maj_np_new = create_synth_data(xxte, yte, minority_label,majority_label,5,lambda_score,'maj_np',pp_group,npp_group)
                    
                        #N(C+,S+) =N(C+,S+)−λ× N(C+,S+)
                        Xtr_min_np_new,Ytr_min_np_new = downsample(xxte, yte, minority_label,majority_label,5,lambda_score,'min_np',pp_group,npp_group)
                        
                        for k in Xtr_min_np_new:
                            Xtr_maj_np_new.append(k)
                        for k in Ytr_min_np_new:
                            Ytr_maj_np_new.append(k)
                        
                        Xtr_1_new = np.array(Xtr_maj_np_new)
                        Ytr_1_new = np.array(Ytr_maj_np_new)
                        
                    
                    added_points = len(Xtr_1_new) - len(xxte)
                    #new batch for new data
                    #split data for validation set
                    Xtr_1_new_split,x_val,Ytr_1_new_split,y_val=train_test_split(Xtr_1_new,Ytr_1_new,test_size=0.10,shuffle=True)
                    
                    data = list(zip(Xtr_1_new_split, Ytr_1_new_split))
                    random.shuffle(data)
                    btach_data = batch_data(data, bs=30)  
                    #fit local model with client's data
                    #create validation data and early stop
                    
                    local_model.fit(btach_data,validation_data=(x_val,y_val),validation_steps=1, callbacks=[e], epochs=3, verbose=0,
                                    class_weight = class_weights)
                    ## find disc score for each client
                    
                    disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model)
                    if disc_score<0:
                
                        pp_group = np_Group
                        npp_group=p_Group
                        disc_score, assigned_positives, total_positives, balanced_accuracy_client = test_client_model(x_test_client, y_test_client,  local_model,pp_group,npp_group)
                    else:
                        pp_group = p_Group
                        npp_group=np_Group
                    
                    
                    trade_off = (1+epsilon**2)*((balanced_accuracy_client*(1-abs(disc_score)))/(epsilon*balanced_accuracy_client+(1-abs(disc_score))))
                    
                    closest_to_zero = min(disc_score, min_disc_score, key=abs)
                    if trade_off>max_trade_off:
                    #if closest_to_zero != min_disc_score:
                        
                        min_disc_score = disc_score
                        max_trade_off = trade_off
                        prev_xtr,prev_ytr = Xtr_1_new, Ytr_1_new
                        points = len(Xtr_1_new) - len(xxte)
                        local_model.save_weights('./checkpoints/law2/income-eqop/my_checkpoint')
                    #if len(prev_disc_score)>0:
                    if len(prev_trade_off)>0:
                        #if disc_score>prev_disc_score[-1]:
                        if trade_off<prev_trade_off[-1]:
                            greater_disc_score+=1
                            print(greater_disc_score)
                            if greater_disc_score >2:
                                local_model.load_weights('./checkpoints/law2/income-eqop/my_checkpoint')
                                added_points = points
                                
                                #Xtr_1_new, Ytr_1_new =prev_xtr,prev_ytr
                                #xxte, yte = Xtr_1_new, Ytr_1_new
                                break
                                
                                
                               
                     
                    #greater_disc_score = 0
                    #lambda_score= lambda_score+0.01
                    #prev_disc_score.append(disc_score)
                    prev_trade_off.append(trade_off)
                    xxte, yte = Xtr_1_new, Ytr_1_new
                    
                
                
                
                xxtee=[]
                ytee=[]
                dataset = tf.data.Dataset.from_tensor_slices((list(prev_xtr), list(prev_ytr)))
                up_dict = {client:batch_data(dataset)}
                clients_batched.update(up_dict)
                #clients_batched[client_name] = batch_data(dataset)
                
                
                
                
                
                
                
        else:
            
            if comm_round!=0:
                Xtr_1_new=np.array(xxte)
                Ytr_1_new=np.array(yte)
                #class_weights = class_weight.compute_class_weight('balanced',
                #                                 np.unique(Ytr_1_new),
                #                                 Ytr_1_new)
                #class_weights = {majority_label:1, minority_label:7}
                class_weights=find_class_Weight(Ytr_1_new, majority_label,minority_label)
                Xtr_1_new,x_val,Ytr_1_new,y_val=train_test_split(Xtr_1_new,Ytr_1_new,test_size=0.10,shuffle=True)
                data = list(zip(Xtr_1_new,Ytr_1_new))
                #random.shuffle(data)
                added_points = 0
                btach_data = batch_data(data, bs=30)
                
                local_model.fit(btach_data,validation_data=(x_val,y_val),validation_steps = 1, callbacks=[e], epochs=3, 
                                verbose=0, class_weight = class_weights)
        
        #scale the model weights and add to list       
        client_names = list(clients_batched.keys())
        bs = list(clients_batched[client_name])[0][0].shape[0]
        #first calculate the total training data points across clinets
        global_count = sum([tf.data.experimental.cardinality(clients_batched[client_name]).numpy() for client_name in client_names])*bs
        global_count = global_count + (added_points)
        # get the total number of data points held by a client
        local_count = len(Xtr_1_new)
        scaling_factor = local_count/global_count
                
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    
    global_model.set_weights(average_weights)
    
    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss,conf,stat_parity,eqop = test_model(X_test, Y_test, global_model, comm_round)
        print("x_test %s "% len(X_test))
        
        TN = conf[0][0]
        FP = conf[0][1]
        FN = conf[1][0]
        TP = conf[1][1]
        sensitivity = TP/(TP+FN) 
        specificity = TN/(FP+TN)
        
        BalanceACC = (sensitivity+specificity)/2
        G_mean = math.sqrt(sensitivity*specificity)
        FN_rate= FN/(FN+TP) 
        FP_rate = FP/(FP+TN) 
        #add the data to arrays
        sensitivity_.append(sensitivity)
        specificity_.append(specificity)
        BalanceACC_.append(BalanceACC)
        G_mean_.append(G_mean)
        FP_rate_.append(FP_rate)
        FN_rate_.append(FN_rate)
        accuracy_.append(global_acc)
        loss_.append(global_loss)
        statistical_parity_.append(stat_parity)
        eqop_.append(eqop)
    end_time.append(((time.time()) - start))

In [ ]:
print(sensitivity_[index])
print(specificity_[index])
print(BalanceACC_[index])
print(G_mean_[index])
print(FP_rate_[index])
print(FN_rate_[index])
print(accuracy_[index])
print(statistical_parity_[index])
print(eqop_[index])
print(statistical_parity_.index(min(statistical_parity_[0:-1], key=abs)))
print(BalanceACC_.index(max(BalanceACC_[0:-1], key=abs)))
print(trade_off.index(max(trade_off[0:-1], key=abs)))
destination = "./results/law/sp/"
client="income-3-clients2-"
output_file = open(destination+client+"-client-bal_acc.txt", "w+")
output_file.write(str(BalanceACC_))
output_file.close()

output_file = open(destination+client+"-client-sen.txt", "w+")
output_file.write(str(sensitivity_))
output_file.close()

output_file = open(destination+client+"-client-spc.txt", "w+")
output_file.write(str(specificity_))
output_file.close()

output_file = open(destination+client+"-client-gmean.txt", "w+")
output_file.write(str(G_mean_))
output_file.close()

output_file = open(destination+client+"-client-fp_rate.txt", "w+")
output_file.write(str(FP_rate_))
output_file.close()

output_file = open(destination+client+"-client-fn_rate.txt", "w+")
output_file.write(str(FN_rate_))
output_file.close()

output_file = open(destination+client+"-client-sp.txt", "w+")
output_file.write(str(statistical_parity_))
output_file.close()

output_file = open(destination+client+"-client-eqop.txt", "w+")
output_file.write(str(eqop_))
output_file.close()
print(index)
#pyplot.plot(BalanceACC_)
plt.plot(BalanceACC_)
plt.title('BalanceACC')
plt.xlabel('Global Round')
plt.ylabel('BalanceACC')
plt.show()
plt.plot(statistical_parity_)
plt.title('statistical_parity')
plt.xlabel('Global Round')
plt.ylabel('statistical_parity_')
plt.show()